# Verma Model

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.varmax import VARMAX
import numpy as np
from statsmodels.tsa.stattools import adfuller
from sklearn import metrics
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")

In [55]:
data = pd.read_csv('Dow Jones Industrial Average Historical Data.csv', parse_dates= True)
data

,Date,Price,Open,High,Low,Vol.,Change %
0,"Dec 31, 2019","28,538.44","28,414.64","28,547.35","28,376.49",193.34M,0.27%
1,"Dec 30, 2019","28,462.14","28,654.76","28,664.69","28,428.98",185.07M,-0.64%
2,"Dec 27, 2019","28,645.26","28,675.34","28,701.66","28,608.98",184.93M,0.08%
3,"Dec 26, 2019","28,621.39","28,539.46","28,624.10","28,535.15",155.97M,0.37%
4,"Dec 24, 2019","28,515.45","28,572.57","28,576.80","28,503.21",95.29M,-0.13%
...,...,...,...,...,...,...,...
2761,"Jan 09, 2009","8,599.18","8,738.80","8,800.45","8,541.75",-,-1.64%
2762,"Jan 08, 2009","8,742.46","8,769.94","8,807.14","8,593.52",-,-0.31%
2763,"Jan 07, 2009","8,769.70","8,996.94","8,996.94","8,690.45",-,-2.72%
2764,"Jan 06, 2009","9,015.10","8,954.57","9,175.19","8,868.07",-,0.69%


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2766 entries, 0 to 2765
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      2766 non-null   object
 1   Price     2766 non-null   object
 2   Open      2766 non-null   object
 3   High      2766 non-null   object
 4   Low       2766 non-null   object
 5   Vol.      2766 non-null   object
 6   Change %  2766 non-null   object
dtypes: object(7)
memory usage: 151.4+ KB


In [56]:
data['Date']= pd.to_datetime(data['Date'])
#data['Price']= pd.to_numeric(data['Price'])
data['Price'] = data['Price'].str.replace(',', '').astype(float)
data['Open'] = data['Open'].str.replace(',', '').astype(float)
data['High']= pd.to_numeric(data['High'].str.replace(',', '').astype(float))
data['Low']= pd.to_numeric(data['Low'].str.replace(',', '').astype(float))
#data['Vol.']= pd.to_numeric(data['Vol.'].str.replace('-', '').astype(string)
data['Vol.']=data['Vol.'].replace('-', 0)
data['Vol.']= pd.to_numeric(data['Vol.'].str.replace('M', '').astype(float))
data['Change %']= pd.to_numeric(data['Change %'].str.replace('%', '').astype(float)) 
data

In [60]:
df=data

In [63]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [64]:
def Augmented_Dickey_Fuller_Test_func(series , column_name):
    print (f'Results of Dickey-Fuller Test for column: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("Data is stationary")
    else:
        print("Conclusion:====>")
        print("Fail to reject the null hypothesis")
        print("Data is non-stationary")

In [59]:
data.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')

In [65]:
for name, column in df[['Open', 'High', 'Low', 'Price']].iteritems():
    Augmented_Dickey_Fuller_Test_func(df[name],name)
    print('\n')

Results of Dickey-Fuller Test for column: Open
Test Statistic                   -1.515300
p-value                           0.525930
No Lags Used                     24.000000
Number of Observations Used    2741.000000
Critical Value (1%)              -3.432738
Critical Value (5%)              -2.862595
Critical Value (10%)             -2.567332
dtype: float64
Conclusion:====>
Fail to reject the null hypothesis
Data is non-stationary


Results of Dickey-Fuller Test for column: High
Test Statistic                   -1.481508
p-value                           0.542639
No Lags Used                     14.000000
Number of Observations Used    2751.000000
Critical Value (1%)              -3.432729
Critical Value (5%)              -2.862591
Critical Value (10%)             -2.567330
dtype: float64
Conclusion:====>
Fail to reject the null hypothesis
Data is non-stationary


Results of Dickey-Fuller Test for column: Low
Test Statistic                   -1.501506
p-value                        

In [66]:
X = df[['Open', 'High', 'Low', 'Price' ]]
train, test = X[0:-30], X[-30:]

In [67]:
train_diff = train.diff()
train_diff.dropna(inplace = True)

In [68]:
for name, column in train_diff[['Open', 'High', 'Low', 'Price' ]].iteritems():
    Augmented_Dickey_Fuller_Test_func(train_diff[name],name)
    print('\n')

Results of Dickey-Fuller Test for column: Open
Test Statistic                -1.191966e+01
p-value                        5.067432e-22
No Lags Used                   2.300000e+01
Number of Observations Used    2.711000e+03
Critical Value (1%)           -3.432764e+00
Critical Value (5%)           -2.862607e+00
Critical Value (10%)          -2.567338e+00
dtype: float64
Conclusion:====>
Reject the null hypothesis
Data is stationary


Results of Dickey-Fuller Test for column: High
Test Statistic                -1.330247e+01
p-value                        7.003189e-25
No Lags Used                   1.800000e+01
Number of Observations Used    2.716000e+03
Critical Value (1%)           -3.432760e+00
Critical Value (5%)           -2.862605e+00
Critical Value (10%)          -2.567337e+00
dtype: float64
Conclusion:====>
Reject the null hypothesis
Data is stationary


Results of Dickey-Fuller Test for column: Low
Test Statistic                -1.208042e+01
p-value                        2.238979e

In [69]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df): 
    res = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = res.lr1
    cvts = res.cvt[:, d[str(1-0.05)]]
    def adjust(val, length= 6): 
        return str(val).ljust(length)
    print('Column Name   >  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), '> ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [70]:
cointegration_test(train_diff[['Open', 'High', 'Low', 'Price']])

Column Name   >  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Open   >  3645.77   > 40.1749   =>   True
High   >  2380.91   > 24.2761   =>   True
Low    >  1214.5    > 12.3212   =>   True
Price  >  368.94    > 4.1296    =>   True


In [71]:
def inverse_diff(actual_df, pred_df):
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns: 
        df_res[str(col)+'_1st_inv_diff'] = actual_df[col].iloc[-1] + df_res[str(col)].cumsum()
    return df_res

In [72]:
from sklearn.model_selection import ParameterGrid
param_grid = {'p': [1,2,3], 'q':[1,2,3], 'tr': ['n','c','t','ct']}
pg = list(ParameterGrid(param_grid))

In [73]:
pg

[{'p': 1, 'q': 1, 'tr': 'n'},
 {'p': 1, 'q': 1, 'tr': 'c'},
 {'p': 1, 'q': 1, 'tr': 't'},
 {'p': 1, 'q': 1, 'tr': 'ct'},
 {'p': 1, 'q': 2, 'tr': 'n'},
 {'p': 1, 'q': 2, 'tr': 'c'},
 {'p': 1, 'q': 2, 'tr': 't'},
 {'p': 1, 'q': 2, 'tr': 'ct'},
 {'p': 1, 'q': 3, 'tr': 'n'},
 {'p': 1, 'q': 3, 'tr': 'c'},
 {'p': 1, 'q': 3, 'tr': 't'},
 {'p': 1, 'q': 3, 'tr': 'ct'},
 {'p': 2, 'q': 1, 'tr': 'n'},
 {'p': 2, 'q': 1, 'tr': 'c'},
 {'p': 2, 'q': 1, 'tr': 't'},
 {'p': 2, 'q': 1, 'tr': 'ct'},
 {'p': 2, 'q': 2, 'tr': 'n'},
 {'p': 2, 'q': 2, 'tr': 'c'},
 {'p': 2, 'q': 2, 'tr': 't'},
 {'p': 2, 'q': 2, 'tr': 'ct'},
 {'p': 2, 'q': 3, 'tr': 'n'},
 {'p': 2, 'q': 3, 'tr': 'c'},
 {'p': 2, 'q': 3, 'tr': 't'},
 {'p': 2, 'q': 3, 'tr': 'ct'},
 {'p': 3, 'q': 1, 'tr': 'n'},
 {'p': 3, 'q': 1, 'tr': 'c'},
 {'p': 3, 'q': 1, 'tr': 't'},
 {'p': 3, 'q': 1, 'tr': 'ct'},
 {'p': 3, 'q': 2, 'tr': 'n'},
 {'p': 3, 'q': 2, 'tr': 'c'},
 {'p': 3, 'q': 2, 'tr': 't'},
 {'p': 3, 'q': 2, 'tr': 'ct'},
 {'p': 3, 'q': 3, 'tr': 'n'},
 {

In [77]:
df_results_moni = pd.DataFrame(columns=['p', 'q', 'tr','RMSE open','RMSE high','RMSE low','RMSE Price'])
print('starting grid search')
start = timer()
for a,b in enumerate(pg):
    p = b.get('p')
    q = b.get('q')
    tr = b.get('tr')
    model = VARMAX(train_diff, order=(p,q), trend=tr).fit()
    z = model.forecast(y=train_diff[['Open', 'High', 'Low', 'Price']].values, steps=30)
    df_pred = pd.DataFrame(z, columns=[ 'Open', 'High', 'Low', 'Price'])
    res = inverse_diff(df[['Open', 'High', 'Low', 'Price']],df_pred)
    openrmse = np.sqrt(metrics.mean_squared_error(test.Open, res.Open_1st_inv_diff))
    highrmse = np.sqrt(metrics.mean_squared_error(test.High, res.High_1st_inv_diff))
    lowrmse = np.sqrt(metrics.mean_squared_error(test.Low, res.Low_1st_inv_diff))
    pricermse = np.sqrt(metrics.mean_squared_error(test.Price, res.Price_1st_inv_diff))
    df_results_moni = df_results_moni.append({'p': p, 'q': q, 'tr': tr,'RMSE open': openrmse,'RMSE high':highrmse,'RMSE low':lowrmse,'RMSE Price':pricermse }, ignore_index=True)
end = timer()
print(f' Total time taken to complete grid search in seconds: {(end - start)}')

starting grid search


KeyboardInterrupt: 

In [78]:
df_results_moni.head()

,p,q,tr,RMSE open,RMSE high,RMSE low,RMSE Price
0,1,1,n,826.927087,795.710959,805.703206,788.459441
1,1,1,c,737.809115,707.584515,729.252793,709.351916
2,1,1,t,663.520083,645.435646,653.100418,645.271570
3,1,1,ct,832.543845,804.685665,850.190037,812.529593
4,1,2,n,826.495787,791.883083,799.758933,796.686734


In [79]:
df_results_moni.sort_values(by = ['RMSE open','RMSE high','RMSE low','RMSE Price'] )

,p,q,tr,RMSE open,RMSE high,RMSE low,RMSE Price
2,1,1,t,663.520083,645.435646,653.100418,645.271570
1,1,1,c,737.809115,707.584515,729.252793,709.351916
5,1,2,c,748.891680,716.195454,729.646043,722.773470
8,1,3,n,817.553774,781.201876,786.604548,779.867562
4,1,2,n,826.495787,791.883083,799.758933,796.686734
0,1,1,n,826.927087,795.710959,805.703206,788.459441
3,1,1,ct,832.543845,804.685665,850.190037,812.529593
7,1,2,ct,890.462245,848.992303,892.345759,856.272686
6,1,2,t,894.486568,845.732335,898.321270,856.416000


In [80]:
df_results_moni.sort_values(by = ['RMSE open','RMSE high','RMSE low','RMSE Price'] )

,p,q,tr,RMSE open,RMSE high,RMSE low,RMSE Price
2,1,1,t,663.520083,645.435646,653.100418,645.271570
1,1,1,c,737.809115,707.584515,729.252793,709.351916
5,1,2,c,748.891680,716.195454,729.646043,722.773470
8,1,3,n,817.553774,781.201876,786.604548,779.867562
4,1,2,n,826.495787,791.883083,799.758933,796.686734
0,1,1,n,826.927087,795.710959,805.703206,788.459441
3,1,1,ct,832.543845,804.685665,850.190037,812.529593
7,1,2,ct,890.462245,848.992303,892.345759,856.272686
6,1,2,t,894.486568,845.732335,898.321270,856.416000


In [81]:
# from above example we can see that p=1 , q=2, tr=n gives least RMSE
model = VARMAX(train_diff[[ 'Open', 'High', 'Low', 'Price' ]], order=(3,3),trends = 'n').fit( disp=False)
result = model.forecast(steps = 30)

KeyboardInterrupt: 

In [ ]:
res = inverse_diff(df[['Open', 'High', 'Low', 'Price' ]],result)
res

In [ ]:
res.head(5)

In [ ]:
for i in ['Open', 'High', 'Low', 'Price' ]:
    print(f'Evaluation metric for {i}')
    timeseries_evaluation_metrics_func(test[str(i)] , res[str(i)+'_1st_inv_diff'])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for i in ['Open', 'High', 'Low', 'Price' ]:
    
    plt.rcParams["figure.figsize"] = [10,7]
    plt.plot( train[str(i)], label='Train '+str(i))
    plt.plot(test[str(i)], label='Test '+str(i))
    plt.plot(res[str(i)+'_1st_inv_diff'], label='Predicted '+str(i))
    plt.legend(loc='best')
    plt.show()